# Essai avec spacy


In [124]:
import numpy as np
import pandas as pd
import json
import glob



In [125]:
import spacy
from spacy.matcher import PhraseMatcher

In [126]:
root_path = '/Volumes/Samsung_T5/JULIETTE/data-CORD'
#astuce : pour trouver facilement le chemin, ouvrir le dossier correspondant 
#et en appuyant sur "alt" aller sur édition et copier le chemin
metadata_path = f'{root_path}/metadata.csv'
meta_df = pd.read_csv(metadata_path)

/Users/juliettemontanteme/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


On code tout d'abord les fonctions de base : ouverture article, renvoi du texte, création de la base des articles ayant un texte. 

In [176]:
def open_article(path):
    file=open(str(root_path)+"/"+str(path),'r')
    article=json.load(file)
    file.close()
    return article

In [128]:
def make_text(article):
    text_list = []
    for entry in article['body_text']:
        text_list.append(entry['text'])
        text_list.append("\n")
    
    text_full=''.join(text_list)
    return text_full

In [129]:
def has_full_text(x):
    if x["pdf_json_files"]==True and x["pmc_json_files"]==True:
        return False
    return True

In [130]:
meta_bool=meta_df[["pdf_json_files","pmc_json_files"]].isnull()
meta_bool["has_full"]=meta_bool.apply(lambda x:has_full_text(x),axis=1)
meta_df_full_text=meta_df.loc[meta_bool["has_full"]]


In [131]:
n = len(meta_df_full_text)
n

68376

On va maintenant passer à la recherche de mots-clés.

Avant de commencer, on effectue un préprocessing rapide des données (on peut ajouter du lemmatizing et enlever les stops words si besoin)

In [162]:
#Chargement de la base de données anglaise
nlp = spacy.blank('en')

In [193]:
#Création de la liste de tous les textes. 
texts = []
i = 0
while i < n:
    path = meta_df_full_text.iloc[i]['pmc_json_files']
    if str(path)[0] != "d":
        i = i + 1
    else : 
        article = open_article(path)
        text = make_text(article)
        texts.append(nlp(text))
        i = i + 1




ValueError: [E088] Text of length 2958441 exceeds maximum of 1000000. The v2.x parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.

In [171]:
path = meta_df_full_text.iloc[51]['pmc_json_files']
print(path)
article = open_article(path)
text = make_text(article)
text

nan


TypeError: can only concatenate str (not "float") to str

In [154]:
doc = nlp(texte0)

In [155]:
matcher = PhraseMatcher(nlp.vocab, attr='LOWER')

In [156]:
hypertension = ['hypertension', 'high blood pressure', 'hypertensive', 'high pressure', 'raised blood pressure', 'htn', 'hbp', 'ht', ]
diabetes = ['diabetes', 'high blood sugar', 'insulin resistance', 'diabetic', 'diabetics', 'dm']
male_gender = ['male gender', 'male', 'gender', 'sex', 'masculine', 'female']
heart_disease = ['cardiopathy', 'heart disease', 'heart', 'chd', 'arrhythmia', 'tachycardia', 'bradycardia', 'fibrillation', 'cardiomyopathy', 'infarction']
COPD_respiratory_system = ['copd', 'emphysema', 'bronchitis', 'asthma', 'bronchiectasis', 'respiratory', 'trachea', 'lung', 'lungs', 'pulmonary']
smoking_status = ['smoking', 'smoke', 'smoker', 'tobacco']
age = ['age', 'old', 'young', 'senior', 'child', 'children']
cerebrovascular_disease = ['cerebrovascular', 'embolism', 'ischemic', 'stroke', 'aneurysm', 'tia', 'subarachnoid']
cancer = ['cancer', 'leukemia', 'cancerology']
kidney_disease = ['kidney', 'edema', 'anemia', 'gfr', 'dialysis']
drinking = ['drinking', 'alcohol', 'alcoholic']
overweight = ['overweight', 'obesity', 'obese', 'bmi']
liver_disease = ['liver', 'fascioliasis', 'cirrhosis', 'hepatitis', 'psc']

In [157]:
respi_tokens_list = [nlp(factor) for factor in COPD_respiratory_system]
matcher.add("RESPI",respi_tokens_list )

In [158]:
matches = matcher(doc)

In [161]:
found_factors = [doc[match[1]:match[2]] for match in matches]
print(found_factors)

[lungs, pulmonary, lung, respiratory, lung, lung, lung, lung, lung, lung, asthma, lung, Asthma, asthma, asthma, asthma, asthma, pulmonary, pulmonary, pulmonary, bronchiectasis, bronchiectasis, lung, lung, lung, Pulmonary, pulmonary, pulmonary, pulmonary, pulmonary, pulmonary, lung, pulmonary, pulmonary, pulmonary, lungs, lung, lung, lung, pulmonary, lung, pulmonary, lung, pulmonary, pulmonary, lungs, pulmonary, pulmonary, pulmonary, pulmonary, lung, pulmonary, pulmonary, pulmonary, pulmonary, pulmonary, pulmonary, pulmonary, lung, pulmonary, lung, lungs, pulmonary, pulmonary, pulmonary, lung, lung, pulmonary, lung, lung, lung, lung, lung, lungs, lungs, lungs, lung, pulmonary, lungs, lung, lung, pulmonary, lungs, pulmonary, pulmonary, pulmonary, lung, pulmonary, pulmonary, pulmonary, pulmonary, pulmonary, lung, pulmonary, lung, pulmonary, lung, respiratory, lung, lung, lung, lung, lung, pulmonary, lung, pulmonary, pulmonary, lung, lung, lung, lung, lung, lung, lung, lung, lung, lung, re

In [111]:
paths =list(meta_df_full_text["pdf_json_files"])



'document_parses/pdf_json/6ff8826ca04c185f3a88ab4b0d5a912aea4ab1a6.json'

['document_parses/pdf_json/d1aafb70c066a2068b02786f8929fd9c900897fb.json',
 'document_parses/pdf_json/6b0567729c2143a66d737eb0a2f63f2dce2e5a7d.json',
 'document_parses/pdf_json/06ced00a5fc04215949aa72528f2eeaae1d58927.json',
 'document_parses/pdf_json/348055649b6b8cf2b9a376498df9bf41f7123605.json',
 'document_parses/pdf_json/5f48792a5fa08bed9f56016f4981ae2ca6031b32.json',
 'document_parses/pdf_json/b2897e1277f56641193a6db73825f707eed3e4c9.json',
 'document_parses/pdf_json/3bb07ea10432f7738413dff9816809cc90f03f99.json',
 'document_parses/pdf_json/5806726a24dc91de3954001effbdffd7a82d54e2.json',
 'document_parses/pdf_json/faaf1022ccfe93b032c5608097a53543ba24aedb.json',
 'document_parses/pdf_json/5b44feca5d6ffaaeb66501fa84cc6dd44d06660a.json']

In [112]:

for path in paths: 
    l = str(path).split(';')
    for new_path in l:
        paths.append(new_path)



KeyboardInterrupt: 

In [96]:
meta_df_full_text["pmc_json_files"][76]


'document_parses/pmc_json/PMC1636417.xml.json'

In [ ]:
list_of_texts =[]
for i in range(n):
    try:
        path = meta_df_full_text["pmc_json_files"][i]
    except KeyError:
        print(f'La clé {i} n existe pas')
    else : 
        try: 
            article=open_article(str(path))
        except FileNotFoundError:
            print(f'{i} and {path}')
        else: 
            text = make_text(article)
            list_of_texts.append(text)



In [86]:

str(meta_df_full_text["pdf_json_files"][78])






'document_parses/pdf_json/52566dccb4bd8044edc87b1a0aa268320a6ea3d4.json; document_parses/pdf_json/8b39433dd865c0f71c7b2f333e1f506b73d722f1.json'

In [76]:
meta_df_full_text["pdf_json_files"][71]

'document_parses/pdf_json/c1ad13d83e926979dbf2bbe52e4944082f28dfea.json'